In [45]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
import shap
from sklearn.multiclass import OneVsRestClassifier 
from tabulate import tabulate
import os

In [46]:
#Make X and y readable for the SVM model
X = pd.read_csv("C:/Users/arm20dh/OneDrive - Florida State University/Desktop/Career/Personal_Project/Raw_Data_Csv_X.csv")
y = pd.read_csv("C:/Users/arm20dh/OneDrive - Florida State University/Desktop/Career/Personal_Project/Raw_Data_Csv_target.csv")
X = X.drop(X.columns[0],axis=1)
y = y.drop(y.columns[0],axis=1)
y = y.values.ravel()
ind = np.arange(999)


column_names = ['column_' + str(i) for i in range(iterations)]
average_compress_test = pd.DataFrame(columns=column_names)
accuracy_list = []
best_percent = []

X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(X, y, ind, test_size=0.2)

#STANDARD SCALER
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#MODEL + ACCURACY
svm_model = SVC(kernel = 'rbf', C = 8, gamma = 'scale')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy_list.append(accuracy)

folder_path = "C:/Users/arm20dh/OneDrive - Florida State University/Desktop/Career/Personal_Project/Preprocessed_Compressed_CSV"
results_df = pd.DataFrame(columns=['Filename', 'Accuracy'])
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path,filename)
        data = pd.read_csv(file_path)
        data = data.drop(data.columns[0],axis=1)
        data = scaler.transform(data)
        data = data[test_indices]
        data_accuracy = accuracy_score(y_test,svm_model.predict(data))
        new_row = pd.DataFrame({'Filename': [filename], 'Accuracy': [data_accuracy]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)
average_compress_test = pd.concat([average_compress_test,results_df['Accuracy']],axis = 1)
best_percent.append((max(average_compress_test.mean(axis=1)-np.mean(accuracy_list))/np.mean(accuracy_list)))
print("The average accuracy from the test set is ",np.mean(accuracy_list))
average_compress_test['average'] = average_compress_test.mean(axis=1)
average_compress_test = average_compress_test.sort_values('average', ascending=False)
print(average_compress_test['average'])
print("The highest percent increase in accuracy is",np.mean(best_percent))

C:\Users\arm20dh\AppData\Local\Temp\ipykernel_22984\1528010683.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


The average accuracy from the test set is  0.66
84    0.685
48    0.675
72    0.675
83    0.675
61    0.675
      ...  
0      0.58
55    0.575
20    0.525
85    0.505
25     0.48
Name: average, Length: 90, dtype: object
The highest percent increase in accuracy is 0.03787878787878791
